In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
import time
import json
import os
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
_txt_  = open('../../data/kaggle_imdb_600k/international-movies-json/03f765ee-8070-449e-a932-e26eef08119f.json').read()
_json_ = json.loads(_txt_)

In [ ]:
class RTJSON(object):
    #
    # __init__()
    #
    def __init__(self, _json_):
        self.fmto_stars = {}
        self.fmto_abs   = {}
        self.absEOL     = {}
        self.starEOL    = {}
        self.__recursiveParseJSON__('$', '$', _json_)

    #
    # __updateFMTO__()
    #
    def __updateFMTO__(self, _parent_, _child_, _abs_):
        if _abs_:
            if _parent_ not in self.fmto_abs: self.fmto_abs[_parent_] = []
            self.fmto_abs[_parent_].append(_child_)
        else:
            if _parent_ not in self.fmto_stars: self.fmto_stars[_parent_] = set()
            self.fmto_stars[_parent_].add(_child_)

    #
    # __updateEOL__() - update end of the line (actual leaf values of the JSON tree structure)
    #
    def __updateEOL__(self, _path_abs_, _path_star_, _value_):
        self.absEOL[_path_abs_] = _value_
        if _path_star_ not in self.starEOL: self.starEOL[_path_star_] = []
        self.starEOL[_path_star_].append(_value_)

    #
    # __recursiveParseJSON__()
    #
    def __recursiveParseJSON__(self, _path_abs_, _path_star_, _json_):
        if type(_json_) == dict:
            for k in _json_.keys():
                _path_abs_child_, _path_star_child_  = _path_abs_+'.'+k, _path_star_+'.'+k
                self.__updateFMTO__(_path_star_, _path_star_child_, False), self.__updateFMTO__(_path_abs_, _path_abs_child_, True)
                self.__recursiveParseJSON__(_path_abs_child_, _path_star_child_, _json_[k])
        elif type(_json_) == list:
            _path_star_child_ = _path_star_+'[*]'
            self.__updateFMTO__(_path_star_, _path_star_child_, False)
            for i in range(len(_json_)):
                _path_abs_child_  = _path_abs_+'['+str(i)+']'
                self.__updateFMTO__(_path_abs_, _path_star_child_, True)
                self.__recursiveParseJSON__(_path_abs_child_, _path_star_child_, _json_[i])
        elif type(_json_) == str:
            self.__updateFMTO__(_path_star_, _path_star_+'-<str>', False), self.__updateFMTO__(_path_abs_, _path_abs_+'-<str>', True)
            self.__updateEOL__(_path_abs_, _path_star_, _json_)
        elif type(_json_) == int:
            self.__updateFMTO__(_path_star_, _path_star_+'-<int>', False), self.__updateFMTO__(_path_abs_, _path_abs_+'-<int>', True)

        elif type(_json_) == float:
            self.__updateFMTO__(_path_star_, _path_star_+'-<float>', False), self.__updateFMTO__(_path_abs_, _path_abs_+'-<float>', True)

        elif type(_json_) == bool:
            self.__updateFMTO__(_path_star_, _path_star_+'-<bool>', False), self.__updateFMTO__(_path_abs_, _path_abs_+'-<bool>', True)

        elif type(_json_) is None:
            self.__updateFMTO__(_path_star_, _path_star_+'-[NONE]', False), self.__updateFMTO__(_path_abs_, _path_abs_+'-[NONE]', True)

        else:
            raise Exception(f'Unknown type ("{type(_json_)}") for ("{_json_}") encountered in parseJSON()')

    #
    # __nodeLabels__()
    #
    def __nodeLabels__(self, _df_):
        _labels_ = {}
        _nodes_  = set(_df_['fm']) | set(_df_['to'])
        for _node_ in _nodes_:
            if   _node_.endswith('[*]'):    _labels_[_node_] = '[*]'
            elif _node_.endswith('<str>')   or \
                 _node_.endswith('<int>')   or \
                 _node_.endswith('<float>') or \
                 _node_.endswith('<bool>'): _labels_[_node_] = _node_[_node_.rindex('<'):]
            elif _node_.endswith('[NONE]'): _labels_[_node_] = '[NONE]'
            elif '.' in _node_:             _labels_[_node_] = _node_[_node_.rindex('.')+1:]
            elif _node_ == '$':             _labels_[_node_] = 'ROOT'
        return _labels_
    #
    # absolutePathGraphDataframe()
    #
    def absolutePathGraphDataframe(self):
        _df_     = rt.graphDictToDataFrame(self.fmto_abs)
        _labels_ = self.__nodeLabels__(_df_)
        return _df_, [('fm','to')], _labels_

    #
    # starGraphDataframe()
    #
    def starPathGraphDataframe(self):
        _df_     = rt.graphDictToDataFrame(self.fmto_stars)
        _labels_ = self.__nodeLabels__(_df_)
        return _df_, [('fm','to')], _labels_


In [ ]:
_rtjson_ = RTJSON(_json_)
_df_, _relates_, _labels_ = _rtjson_.starPathGraphDataframe()
_g_ = rt.createNetworkXGraph(_df_, _relates_)
_pos_ = rt.hyperTreeLayout(_g_)
rt.linkNode(_df_, _relates_, _pos_, node_labels=_labels_, node_labels_only=True, node_size='small', draw_labels=True, w=1536, h=768)